### 전처리

In [ ]:
# 필요한 라이브러리 로드
import os
import pandas as pd
import xlwings as xw

In [ ]:
# 'adobe_raw' 디렉토리에서 처리할 원본 파일 탐색
# 'New_Weekly_캠페인_방문_Ken'으로 시작하는 파일 사용
for filename in os.listdir('./adobe_raw/'):
    if filename.startswith('New_Weekly_캠페인_방문_Ken'):
        campaign_visit_file = './adobe_raw/' + filename
        break

In [ ]:
# xlwings로 Excel 파일을 열고 데이터 읽기
# DRM 보호 CSV 파일을 읽기 위해 Excel 앱을 직접 제어하는 xlwings 사용
# try...finally 구문으로 Excel 앱의 확실한 종료 보장
workbook = None
try:
    workbook = xw.Book(campaign_visit_file)
    sheet1 = workbook.sheets[0].used_range.value
    df_campaign_visit = pd.DataFrame(sheet1)
finally:
    if workbook:
        workbook.app.quit()

In [ ]:
# 컬럼명 변경
df_campaign_visit.columns = ['LastTouchChannel', 'WeekNumber','Entries']

In [ ]:
# 리포트 헤더 부분(상위 8행) 제외
# .copy()를 사용하여 SettingWithCopyWarning 방지
df_campaign_visit = df_campaign_visit.iloc[8:].copy()

In [ ]:
# 구분자(#######) 행 삭제
df_campaign_visit = df_campaign_visit[df_campaign_visit['LastTouchChannel'] != '##############################################']

In [ ]:
# CampaignName 컬럼 생성 및 값 채우기
# Adobe Analytics 리포트 형식에 맞춰 캠페인명을 채워넣는 로직
# '#'을 포함하는 행의 값을 campaign_name으로 간주하고, 그 아래 행들에 해당 값을 복사
df_campaign_visit['CampaignName'] = df_campaign_visit['LastTouchChannel']

# '#'을 포함하는 행만 남기고 나머지는 NaN으로 처리한 후, ffill로 빈 값을 채움
campaign_names = df_campaign_visit['CampaignName'].where(df_campaign_visit['CampaignName'].str.contains('#', na=False))
df_campaign_visit['CampaignName'] = campaign_names.ffill()

In [ ]:
# 불필요한 행 및 결측치 제거
df_campaign_visit = df_campaign_visit.dropna(subset=['LastTouchChannel'])
df_campaign_visit = df_campaign_visit[df_campaign_visit['LastTouchChannel'] != 'Last Touch Channel']

In [ ]:
# 데이터 타입 변경
# pd.to_numeric과 .fillna(0)를 사용하여 안정적인 타입 변환
df_campaign_visit['Entries'] = pd.to_numeric(df_campaign_visit['Entries'], errors='coerce').fillna(0).astype(int)

In [ ]:
# csv 파일로 저장
df_campaign_visit.to_csv('./csv_raw/campaign_visit.csv', index=False, encoding='utf-8-sig')